# Homework 1 - Streaming (15 pts)

Please turn in your note book naming as **BDM\_HW1\_NetID.ipynb**.


## Task 1 (5 points)

You are asked to write a Python generator that takes a sequence of characters as input (think of it as a string iterator that can only be traversed forward) and returns a generator of substrings containing consecutive characters in the string. For example, substrings of consecutive characters in `'Good morning !!!'` are `'oo'` and `'!!!'`. Your task is to complete the generator below (only fill in the body of **`findDuplicateSequence()`**).

In [ ]:
def findDuplicateSequence(s):
    # YOU CAN ONLY EDIT BELOW THIS LINE
    result = filter(lambda x: len(x)>1, reduce(lambda x,y: x+y if x[-1] == y else x+','+y,s,',').split(','))
    return result
from functools import reduce

In [ ]:

# DO NOT EDIT THIS CELL, YOU CAN USE THIS TO CHECK THE CODE IN THE PREVIOUS CELL

print(repr(next(findDuplicateSequence(iter('aabccccedaaa'))))) # 'aa'
print(list(findDuplicateSequence(iter('aabccccedaaa')))) # 'aa', 'cccc', 'aaa'
print(list(findDuplicateSequence(iter('Good morning!!!')))) # 'oo', '!!!'

'aa'
['aa', 'cccc', 'aaa']
['oo', '!!!']



## Task 2 (10 points)

We are greatly inspired by the [Consumer Complaints](https://github.com/InsightDataScience/consumer_complaints) challenge from [InsightDataScience](https://github.com/InsightDataScience/). In fact, we are going to tackle the same challenge but streaming. Please read through the challenge at the following link (the most important sections are “Input dataset” and “Expected output”):

<https://github.com/InsightDataScience/consumer_complaints>

Our problem is different than the one above in that the INPUT is also <ins>sorted by product (alphabetically) and year (ascending)</ins>, similar to the requested output format. You are asked to complete the generator below to produce the expected output. You should not edit any other code.

### INPUT
Your code will be evaluated against the original data set (in CSV format) downloaded from the below link (of course with a sorted version). For your convenience, a smaller version of the file is provided below for testing purposes.

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data



In [ ]:
!gdown --id 1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam -O complaints.csv
!head -n 1 complaints.csv

Downloading...
From: https://drive.google.com/uc?id=1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam
To: /content/complaints.csv
100% 3.91M/3.91M [00:00<00:00, 166MB/s]
Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID


In [ ]:
!head -n 2 complaints.csv
!pip install mrjob
!gdown --id 1sq4-zXn2Z82mdLSBBegEgsUsfqtgza-C -O mapreduce.py

Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
2015-12-31,Bank account or service,Checking account,"Making/receiving payments, sending money",,,,FIRSTBANK PUERTO RICO,PR,00902,Older American,N/A,Referral,2016-02-04,Closed with explanation,Yes,No,1723943
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1sq4-zXn2Z82mdLSBBegEgsUsfqtgza-C 



In [ ]:
import csv
import mapreduce as mr

In [ ]:
def task2(reader):
  # YOU CAN ONLY EDIT THE CODE BELOW THIS LINE
  def mapper1(_, row):
    product_time_company = row['Product'].lower(), row['Date received'].split('-')[0], row['Company']
    yield (product_time_company,1)
  def reducer1(product_time_company, company_time_counts):
    yield (product_time_company,sum(company_time_counts))

  def mapper2(product_time_company, counts):#total number of complaints received for that product and year
      yield ((product_time_company[0], product_time_company[1]), counts)
  def reducer2(product_time, product_counts):
      yield (product_time,sum(product_counts))

  def mapper3(prodcut_time_company, counts):#total number of companies receiving at least one complaint for that product and year
      yield ((prodcut_time_company[0], prodcut_time_company[1]), 1)
  def reducer3(product_time, counts):
      yield (product_time, sum(counts))

  def mapper4(prodcut_time_company, counts):#max
    yield ((prodcut_time_company[0], prodcut_time_company[1]), (prodcut_time_company [2], counts))
  def reducer4(prodcut_time, company_counts):
    yield (prodcut_time, (max(company_counts, key=lambda x: x[1])))

  row = enumerate(reader)
  output1 = list(mr.run(row, mapper1, reducer1))
  output2 = list(mr.run(output1, mapper2, reducer2))
  output3 = list(mr.run(output1, mapper3, reducer3))
  output4 = list(mr.run(output1, mapper4, reducer4))
  
  result = []
  for i in range(len(output3)):
    result.append(output4[i][0] + (output3[i][1],) + (output2[i][1],) +(int(output4[i][1][1] / output2[i][1]*100),))
  return result

  pass

In [ ]:
# DO NOT EDIT THIS CELL, YOU CAN RUN TO QUICKLY CHECK YOUR RESULTS

import csv

with open('complaints.csv', 'r') as fi, open('output.csv', 'w') as fo:
  reader = csv.DictReader(fi)
  writer = csv.writer(fo)
  for row in task2(reader):
    writer.writerow(row)

!head output.csv

bank account or service,2015,1,1,100
bank account or service,2016,2,2,50
checking or savings account,2017,1,1,100
checking or savings account,2018,10,20,25
checking or savings account,2019,72,461,13
checking or savings account,2020,3,3,33
consumer loan,2015,1,1,100
consumer loan,2016,1,1,100
consumer loan,2017,1,1,100
credit card,2016,4,4,25
